# PyTorch-learning "Hello World"-FashionMNIST图像分类

PyTorch官方推荐的"Hello World!"项目，让我们走一遍深度学习的完整流程
- 加载数据
- 指定设备
- 定义模型
- 定义损失函数和优化器
- $Train$
- $Test$
- $Save$

In [1]:
import os, sys
sys.path.append(os.pardir)
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

## 加载数据

`PyTorch`有两个处理数据的核心组件，一个是`Dataset`, 一个是`DataLoader`  
`Dataset`: 存储(样本, 标签)对  
`DataLoader`: 从Dataset中取数据+打包管理

In [ ]:
training_data = datasets.FashionMNIST(
    root = "../Dataset",
    train=True,
    download=False,
    transform=ToTensor(),
)

test_data = datasets.FashionMNIST(
    root="../Dataset",
    train=False,
    download=False,
    transform=ToTensor(),
)

batch_size = 64

train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

"""
for X, y in test_dataloader:
    print(f"Shape of X : {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")

X : [batch_size, channels, height, width]
y : [batch_size]
"""

## 指定设备

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"

## 定义模型

In [4]:
class NeuralNetWork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
    
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetWork().to(device)

## 定义损失函数和优化器

In [5]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

## $Train$

In [6]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # 计算损失函数值
        pred = model(X)
        loss = loss_fn(pred, y)

        # 反向传播
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")


## $Test$

In [7]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100 * correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


***

In [8]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t + 1}\n------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done")

Epoch 1
------------------------------
loss: 2.317452 [   64/60000]
loss: 2.301095 [ 6464/60000]
loss: 2.288608 [12864/60000]
loss: 2.271758 [19264/60000]
loss: 2.254943 [25664/60000]
loss: 2.229294 [32064/60000]
loss: 2.231590 [38464/60000]
loss: 2.204962 [44864/60000]
loss: 2.204659 [51264/60000]
loss: 2.167528 [57664/60000]
Test Error: 
 Accuracy: 42.8%, Avg loss: 2.166876 

Epoch 2
------------------------------
loss: 2.183429 [   64/60000]
loss: 2.171997 [ 6464/60000]
loss: 2.123540 [12864/60000]
loss: 2.126591 [19264/60000]
loss: 2.089655 [25664/60000]
loss: 2.035301 [32064/60000]
loss: 2.044190 [38464/60000]
loss: 1.980362 [44864/60000]
loss: 1.986578 [51264/60000]
loss: 1.907993 [57664/60000]
Test Error: 
 Accuracy: 53.0%, Avg loss: 1.911174 

Epoch 3
------------------------------
loss: 1.949002 [   64/60000]
loss: 1.918992 [ 6464/60000]
loss: 1.809729 [12864/60000]
loss: 1.830220 [19264/60000]
loss: 1.751012 [25664/60000]
loss: 1.694007 [32064/60000]
loss: 1.686216 [38464/600

## 保存模型

In [9]:
torch.save(model.state_dict(),"../models/FashionMNIST/FashionMNIST.pth")
print("Saved PyTorch Model")

Saved PyTorch Model


## 加载保存的模型

In [10]:
model = NeuralNetWork().to(device)
model.load_state_dict(torch.load("../models/FashionMNIST/FashionMNIST.pth"))

C:\Users\30385\AppData\Local\Temp\ipykernel_16200\4103798215.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("../models/FashionMNIST/Fas

<All keys matched successfully>

## 测试

In [11]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
